In [3]:
import pickle
import math
import numpy as np
import time
import networkx as nx

import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [4]:
AutoML_path = ""
log_folder = "alphad3m_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022'

combination_name = 'DDN+GIN'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today

In [5]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))



39
7811


In [6]:
from torch_geometric.data import Data
from collections import defaultdict
device = torch.device('cpu')
graphs = defaultdict(list)

for pipeline in pipeline_steps_class:
    data_name = pipeline[0]
    if data_name in DDN:
        g = pipeline[2]
        score = pipeline[3]
        node_attribute_dict = nx.get_node_attributes(g, name='feature')
        node_attribute_list = [np.hstack((node_attribute_dict[i],DDN[data_name])) for i in node_attribute_dict]
#         node_attribute_list = [np.hstack((node_attribute_dict[i],X_meta[data_name])) for i in node_attribute_dict]
        node_x = torch.tensor(node_attribute_list, dtype=torch.float)

        edge_list = list(g.edges())
        edge_index = torch.tensor(edge_list,dtype=torch.long).t()

        graph_data = Data(edge_index=edge_index, x=node_x, y=score)
        graphs[data_name].append(graph_data.to(device))
        

<ipython-input-6-30d66257d9b8>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  node_x = torch.tensor(node_attribute_list, dtype=torch.float)


In [7]:
print(len(graphs))
graphs['adult'][0]

39


Data(x=[36, 37], edge_index=[2, 15], y=0.2895)

In [8]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.nn import global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GIN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GINConv(torch.nn.Sequential(
           Linear(in_channels, hidden_channels)
        ))
        self.conv2 = GINConv(torch.nn.Sequential(
           Linear(hidden_channels, hidden_channels)
        ))
        self.conv3 = GINConv(torch.nn.Sequential(
           Linear(hidden_channels, hidden_channels)
        ))
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        dataset_embedding = x[:, 40:104]
#         print(x.shape, dataset_embedding.shape)
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
#         x = torch.cat((x, dataset_embedding), dim=1)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.2, training=self.training)
#         print(x.shape, dataset_embedding.shape)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        x = self.lin3(x)
        
        return x

In [10]:
from torch_geometric.data import DataLoader

target_test_pred_time = {}
data_names = list(DDN.keys())
data_embed = DDN
batch_size = 64

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        print("-"*40, threshold)
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print("+"*30)
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        graphs_train = []
        graphs_test = []
        for key in graphs:
            if key in task_train:
                graphs_train += graphs[key]
            elif key in task_test:
                graphs_test += graphs[key]
        train_loader = DataLoader(graphs_train, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(graphs_test, batch_size=batch_size, shuffle=False)
        print("graphs_train shape:", len(graphs_train), "graphs_train[0] shape:", graphs_train[0])
        print("graphs_test shape:", len(graphs_test), "graphs_test[0] shape:", graphs_test[0])

        model = GIN(in_channels=graphs_train[0].num_node_features, hidden_channels=64).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = torch.nn.MSELoss() 
        def train():
            model.train()
            for data in train_loader:  # Iterate in batches over the training dataset.
                out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
                loss = criterion(out, data.y.reshape(data.y.shape[0],1).to(device))  # Compute the loss.
                # loss = criterion(out, data.y.to(device))  # Compute the loss.
                loss.backward()  # Derive gradients.
                optimizer.step()  # Update parameters based on gradients.
                optimizer.zero_grad()  # Clear gradients.

        def test(loader):
            model.eval()
            test = []
            pred = []
            for data in loader:  # Iterate in batches over the training/test dataset.
                out = model(data.x, data.edge_index, data.batch)  
        #         print(out, data.y)
                if len(test) == 0:
                    test = data.y
                    pred = out.detach()
                else:
                    test = torch.cat((test, data.y))
                    pred = torch.cat((pred, out.detach()))
            return test, pred, mean_absolute_error(test.cpu(), pred.cpu()), mean_squared_error(test.cpu(), pred.cpu())  # Derive ratio of correct predictions.


        start = time.time()
        for epoch in range(10):
            train()
            _, _, train_mae, train_mse = test(train_loader)
            test_y, pred_y, test_mae, test_mse = test(test_loader)
        #     test_acc = test(test_loader)
        end  = time.time()
        duration = end-start
        print(f'Epoch: {epoch:03d}, Train MAE: {train_mae:.4f}, Train MSE: {train_mse:.4f}')
        print(f'Epoch: {epoch:03d}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}')
        print("Time:", duration)
        
        target_test_pred_time[dict_key] = (test_y, pred_y, duration, test_mae, test_mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
---------------------------------------- 0.05
graphs_train shape: 1526 graphs_train[0] shape: Data(x=[36, 37], edge_index=[2, 14], y=0.7580213903743316)
graphs_test shape: 452 graphs_test[0] shape: Data(x=[36, 37], edge_index=[2, 15], y=0.2895)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1414, Train MSE: 0.0281
Epoch: 009, Test MAE: 0.1150, Test MSE: 0.0201
Time: 3.1106350421905518
---------------------------------------- 0.1
graphs_train shape: 3405 graphs_train[0] shape: Data(x=[36, 37], edge_index=[2, 14], y=0.7580213903743316)
graphs_test shape: 452 graphs_test[0] shape: Data(x=[36, 37], edge_index=[2, 15], y=0.2895)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1264, Train MSE: 0.0238
Epoch: 009, Test MAE: 0.1073, Test MSE: 0.0193
Time: 6.3311591148376465
---------------------------------------- 0.15000000000000002
graphs_train shape: 5415 graphs_train[0] shape: Data(x=[36, 37], edge_index=[2, 14], y=0.7580213903743316)
graphs_test shape: 452 graphs_test[0] shape: Data(x=[36, 37], edge_index=[2, 15], y=0.2895)


...

Epoch: 009, Train MAE: 0.1453, Train MSE: 0.0429
Epoch: 009, Test MAE: 0.3244, Test MSE: 0.1133
Time: 10.745970010757446
---------------------------------------- 0.35000000000000003
graphs_train shape: 6884 graphs_train[0] shape: Data(x=[36, 37], edge_index=[2, 14], y=0.7580213903743316)
graphs_test shape: 216 graphs_test[0] shape: Data(x=[36, 37], edge_index=[2, 12], y=0.4497787212787213)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1605, Train MSE: 0.0521
Epoch: 009, Test MAE: 0.3367, Test MSE: 0.1214
Time: 13.124389171600342
---------------------------------------- 0.4
graphs_train shape: 7595 graphs_train[0] shape: Data(x=[36, 37], edge_index=[2, 14], y=0.7580213903743316)
graphs_test shape: 216 graphs_test[0] shape: Data(x=[36, 37], edge_index=[2, 12], y=0.4497787212787213)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1654, Train MSE: 0.0534
Epoch: 009, Test MAE: 0.3541, Test MSE: 0.1334
Time: 14.381028175354004
---------------------------------------- 0.45
++++++++++++++++++++++++++++++
volkert_DDN+GIN_0.4 volkert_DDN+GIN_0.45
38 38
